In [23]:
import os 
import pandas as pd
import numpy as np
import plotly.express as px
import math

import cleaning as c
import importlib
importlib.reload(c)

import warnings
warnings.filterwarnings("ignore")


In [24]:
url = "../Data/sampleData.csv"
df = pd.read_csv(url)
df.head()

,date,text,userid_str,status_id_str,id,name,birth,term_partisanship,term_type,term_state,country,Bucket,SentimentScore,version
0,2015-09-28,I commend all of law enforcement for the secur...,156333623.0,6.485341e+17,3517566,Al Green,1947-09-01,Democrat,rep,TX,China,2 or 3,NaN,2019_batch1
1,2019-02-24,"Socialist tyrant Maduro starves, tortures his ...",235312723.0,1.099734e+18,3559992,Scott DesJarlais,1964-02-21,Republican,rep,TN,China,1,2.0,2019_batch1
2,2019-02-24,"Socialist tyrant Maduro starves, tortures his ...",235312723.0,1.099734e+18,3559992,Scott DesJarlais,1964-02-21,Republican,rep,TN,Iran,2 or 3,NaN,2019_batch1
3,2014-06-24,Inhofe: The Canadians have already had convers...,20546536.0,4.815273e+17,3468157,James M. Inhofe,1934-11-17,Republican,sen,OK,Canada,2 or 3,NaN,2019_batch1
4,2015-07-14,Statement on Iran nuclear agreement. http://t....,18773159.0,6.210401e+17,3833428,Scott Perry,1962-05-27,Republican,rep,PA,Iran,2 or 3,NaN,2019_batch1


In [25]:
df["year"] = df["birth"].apply(c.get_year)
df["term_state"] = df["term_state"].apply(c.clean_state)

In [26]:
bin_width= 10
nbins = math.ceil((df["year"].max() - df["year"].min()) / bin_width)
px.histogram(df, x = "year", nbins = nbins)

In [30]:
df["term_partisanship"] = df["term_partisanship"].apply(c.clean_state)
df["term_partisanship"].unique()

dems = df[df["term_partisanship"] == "Democrat"]
reps = df[df["term_partisanship"] == "Republican"]
othr = df[df["term_partisanship"].isin(["Independent", "Unknown"])]

In [34]:
bin_width= 10
nbins_dems = math.ceil((dems["year"].max() - dems["year"].min()) / bin_width)
nbins_reps = math.ceil((reps["year"].max() - reps["year"].min()) / bin_width)
nbins_othr = math.ceil((othr["year"].max() - othr["year"].min()) / bin_width)

px.histogram(reps, x = "year", nbins = nbins_reps)

In [39]:
state_v_country = pd.crosstab(df["term_state"], df["country"])
state_v_country["total"] = state_v_country["Canada"] + state_v_country["China"] + state_v_country["Iran"]
state_v_country.reset_index(inplace = True)
state_v_country.head()

country,term_state,Canada,China,Iran,total
0,AK,40,98,16,154
1,AL,9,142,21,172
2,AR,21,540,90,651
3,AZ,28,249,31,308
4,CA,95,692,161,948


In [40]:
import plotly.graph_objects as go

In [42]:
fig = go.Figure(
    data=[
        go.Bar(name='Canada', x = state_v_country["term_state"], y = state_v_country["Canada"], yaxis='y', offsetgroup=1),
        go.Bar(name='China', x = state_v_country["term_state"], y = state_v_country["China"], yaxis='y', offsetgroup=2),
        go.Bar(name='Iran', x = state_v_country["term_state"], y = state_v_country["Iran"], yaxis='y', offsetgroup=3),
    ],
    layout={
        'yaxis': {'title': '# of Mentions by State'}
    },
)

# Change the bar mode
fig.update_layout(title_text = "# of Tweets For Each Country By State", title_x = 0.5, barmode='group')
fig.show()